In [ ]:
data_path = '/data/desponds/data/Classification'
%load_ext autoreload
%autoreload 2

In [ ]:
from preprocessing import preprocessing_review_classification
datasets, tokenized = preprocessing_review_classification()

In [ ]:
from training import get_trainers_review_classification
trainers = get_trainers_review_classification(data_path, tokenized, langs = ['en'])

In [ ]:
from training import get_models_review_classification
models = get_models_review_classification(data_path, trainers)

## Predictions

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base', cache_dir="/data/desponds/.cache")

def preprocess_after_trad(dataset, no_remove = False):
    def tokenize_function(examples):
        return tokenizer(examples["text"], 
                         padding="max_length", truncation=True)
    if not no_remove :
        dataset = dataset.remove_columns(['input_ids', 'attention_mask'])
    dataset = dataset.map(tokenize_function)
    return dataset

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /data/desponds/.cache/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /data/desponds/.cache/models--roberta-

In [6]:
import pickle 
dataset,results = {}, {}
#Basecase : no translation
dataset[f"translated_0_4"] = preprocess_after_trad(tokenized['fr']['test'])
results[f"translated_0_4"] = models['en'].predict(dataset[f'translated_0_4'])

for i in range(1,4+1):
    with open(f'/data/desponds/data/Classification/translated_dataset/translated_epoch_1_{i}_4.pickle', 'rb') as handle:
        dataset[f"translated_{i}_4"] = pickle.load(handle)
    dataset[f"translated_{i}_4"] = preprocess_after_trad(dataset[f"translated_{i}_4"])
    results[f"translated_{i}_4"] = models['en'].predict(dataset[f'translated_{i}_4'])

with open(f'/data/desponds/data/Classification/translated_dataset/translated_epoch_5_4_4.pickle', 'rb') as handle:
        dataset[f"translated_4_4_epoch_5"] = pickle.load(handle)    
dataset[f"translated_4_4_epoch_5"] = preprocess_after_trad(dataset[f"translated_4_4_epoch_5"])
results[f"translated_4_4_epoch_5"] = models['en'].predict(dataset[f"translated_4_4_epoch_5"])

Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/fr/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-b3f66e1ca092068a.arrow
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5000
  Batch size = 16
/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [7]:
final_results = { k: v[2]['test_accuracy'] for k,v in results.items()}
final_results['Helsinki_translator'] = 0.552
final_results

{'translated_4_4_epoch_5': 0.4608, 'Helsinki_translator': 0.552}

In [8]:
final_results = {
 'translated_0_4': 0.2998,
 'translated_1_4': 0.4618,
 'translated_2_4': 0.4592,
 'translated_3_4': 0.4646,
 'translated_4_4': 0.4468,
 'translated_4_4_epoch_5': 0.4608,
 'Helsinki_translator': 0.552}
final_results

{'translated_0_4': 0.2998,
 'translated_1_4': 0.4618,
 'translated_2_4': 0.4592,
 'translated_3_4': 0.4646,
 'translated_4_4': 0.4468,
 'translated_4_4_epoch_5': 0.4608,
 'Helsinki_translator': 0.552}